In [81]:
import os
from timeit import default_timer as timer
from glob import glob
import pandas as pd
import numpy as np

In [127]:
country_code = "US"
if os.getenv('CLUSTER')=='PRINCE':
    path_to_data='/scratch/spf248/twitter/data'
else:
    path_to_data='../../data'

In [125]:
user_location_2_geo_id=pd.read_csv(os.path.join(path_to_data,'official','urban',country_code,'user_location_2_geo_id.csv'),index_col=0)
user_location_2_geo_id=user_location_2_geo_id[['user_location','metro_area_name']].sort_values(by=['user_location']).reset_index(drop=True)
user_location_2_geo_id.to_csv(os.path.join(path_to_data,'official','urban',country_code,'user_location_2_metro_area.csv'))

In [83]:
if country_code!='US':
    labor_market_stats=pd.read_csv(os.path.join(path_to_data,'official','urban',country_code,'labor_market_stats.csv'),sep=';')
    labor_market_stats.rename(columns={'value':'unemployment_rate','NAME':'metro_area_name'},inplace=True)
    labor_market_stats['date']=pd.to_datetime(labor_market_stats.apply(lambda x:str(x['year'])+'-Q'+str(x['quarter']),1)) + pd.offsets.QuarterEnd(0)
    labor_market_stats['month'] = labor_market_stats['date'].apply(lambda x:x.month)
    labor_market_stats = labor_market_stats[['year','month','metro_area_name','unemployment_rate']].sort_values(
    by=['year','month','metro_area_name']).reset_index(drop=True)
    if country_code =='CO':
        labor_market_stats.unemployment_rate=labor_market_stats.unemployment_rate.apply(lambda x:np.float(x.replace(',','.'))/100)
    labor_market_stats.to_csv(os.path.join(path_to_data,'official','urban',country_code,'time_series_unemployment_rate.csv'))

# US - compute average rate across counties

In [142]:
labor_market_stats=pd.read_csv(os.path.join(path_to_data,'official','urban',country_code,'labor_market_stats.csv'),index_col=0)
pop_metro_area=pd.read_csv(os.path.join(path_to_data,'official','urban',country_code,'pop_metro_area.csv'),index_col=0)

/Users/samuelfraiberger/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [187]:
pop_metro_area=pop_metro_area.groupby(['metro_area_name','geo_id'],as_index=False)['pop'].mean()
df=labor_market_stats.merge(pop_metro_area,on='geo_id')
df['unemployment_rate']=df['unemployment_rate'].multiply(df['pop'])
df=df.groupby(['year','month','metro_area_name']).sum()
df=df['unemployment_rate'].divide(df['pop']).rename('unemployment_rate')
df/=100
df=df.to_frame().reset_index()
df=df.sort_values(by=['year','month','metro_area_name']).reset_index(drop=True)
df.to_csv(os.path.join(path_to_data,'official','urban',country_code,'time_series_unemployment_rate.csv'))